<a href="https://colab.research.google.com/github/msaadsadiq/BigDataCourse/blob/master/Assignment_1_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECE 795 - Big Data
## Assignment #1 - Querying Google Cloud BigQuery using SQL


### Before you begin


1.   Create your own project using using the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) if you do not already have one.
2.   [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
3.   [Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) APIs for the project. (should be already enabled)
  

### Provide your credentials to the runtime

In [3]:
# Authenticate your student profile

from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## There are several ways to access your BigQuery tables 
1. Magics command
2. Google-cloud-bigquery
3. Pandas dataframe 


### Set the Project ID 

In [0]:
project_id = 'ece795'

### We will use the GSOD sample table contains weather information collected by NOAA, such as precipitation amounts and wind speeds from late 1929 to early 2010.

### 1. Magics library method


In [15]:
# The google.cloud.bigquery library also includes a magic command which runs a query and displays the result, optionally saving it to a variable as a DataFrame.

%%bigquery --project ece795 df
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`


# This command runs a query and displays the result, optionally saving it to a variable as a `DataFrame`.

,total_rows
0,114420316


### 2. google-cloud-bigQuery method

In [21]:
# 2. google-cloud-bigQuery method

from google.cloud import bigquery

client = bigquery.Client(project=project_id)

row_count = client.query('''
  SELECT 
    COUNT(*) as total
  FROM `bigquery-public-data.samples.gsod`''').to_dataframe().total[0]


print('Full dataset has %d rows' % row_count)

Full dataset has 114420316 rows


### 3. Pandas data frame

In [23]:

# The pandas-gbq library is a community led project by the pandas community. It covers basic functionality, such as writing a DataFrame to BigQuery and running a query, but as a third-party library it may not handle all BigQuery features or use cases.

import pandas as pd

sample_count = 2000
row_count = pd.io.gbq.read_gbq('''
  SELECT COUNT(*) as total
  FROM `bigquery-public-data.samples.gsod`''', 
                           project_id=project_id, verbose=False, dialect='standard')

print(row_count)

       total
0  114420316


# Lets try some sample Queries on GSOD data

### Sample 2000 random rows from the full Table

In [25]:
# Using Google Cloud method

from google.cloud import bigquery

client = bigquery.Client(project=project_id)

sample_count = 2000
row_count = client.query('''
  SELECT 
    COUNT(*) as total
  FROM `bigquery-public-data.samples.gsod`''').to_dataframe().total[0]

df_gc = client.query('''
  SELECT
    *
  FROM
    `bigquery-public-data.samples.gsod`
  WHERE RAND() < %d/%d
''' % (sample_count, row_count)).to_dataframe()

# top 10 rows
df_gc.head(10)


,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,...,min_temperature,min_temperature_explicit,total_precipitation,snow_depth,fog,rain,snow,hail,thunder,tornado
0,724067,99999,1984,11,28,56.299999,15,49.700001,15.0,NaN,...,None,None,0.00,NaN,False,False,False,False,False,False
1,154980,99999,1991,12,14,32.599998,12,NaN,NaN,NaN,...,None,None,0.00,NaN,False,False,False,False,False,False
2,10490,99999,2001,4,18,32.799999,20,27.299999,20.0,NaN,...,None,None,0.08,NaN,False,False,False,False,False,False
3,804100,99999,2002,1,9,78.099998,11,60.799999,11.0,NaN,...,None,None,0.00,NaN,False,False,False,False,False,False
4,142531,99999,2006,1,28,49.000000,7,45.400002,7.0,NaN,...,None,None,NaN,NaN,False,False,False,False,False,False
5,125800,99999,2010,2,4,25.799999,24,22.799999,24.0,NaN,...,None,None,0.00,11.0,True,True,True,True,True,True
6,555690,99999,1988,10,12,50.799999,4,19.100000,4.0,NaN,...,None,None,0.00,NaN,False,False,False,False,False,False
7,60960,99999,1985,1,28,21.299999,4,19.500000,4.0,1014.599976,...,None,None,0.00,NaN,False,False,False,False,False,False
8,612960,99999,1964,10,21,81.000000,6,69.000000,6.0,1011.299988,...,None,None,0.00,NaN,False,False,False,False,False,False
9,334460,99999,1977,11,14,37.400002,7,35.299999,7.0,1003.500000,...,None,None,0.39,NaN,False,False,False,False,False,False


### Describe the data using GCloud *method*

In [26]:
df_gc.describe()

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,...,num_mean_station_pressure_samples,mean_visibility,num_mean_visibility_samples,mean_wind_speed,num_mean_wind_speed_samples,max_sustained_wind_speed,max_gust_wind_speed,max_temperature,total_precipitation,snow_depth
count,2024.000000,2024.000000,2024.000000,2024.000000,2024.000000,2024.000000,2024.00000,1932.000000,1932.000000,1553.000000,...,734.000000,1814.000000,1814.000000,2001.000000,2001.000000,1980.000000,263.000000,2020.000000,1841.000000,99.000000
mean,513678.177372,91209.933794,1987.038043,6.614625,15.937253,52.714130,13.14081,42.268841,13.099379,1014.729684,...,12.166213,12.165160,12.750827,6.952424,13.112444,12.282172,24.860076,44.111139,0.069565,8.992929
std,302377.222645,25187.413741,16.521710,3.435833,8.683227,24.065121,7.86631,22.589160,7.903841,9.342748,...,7.828632,9.433664,7.794074,5.023333,7.848876,6.729327,8.916061,24.025859,0.392102,9.077504
min,10015.000000,8.000000,1935.000000,1.000000,1.000000,-99.099998,4.00000,-105.699997,4.000000,964.000000,...,4.000000,0.000000,4.000000,0.000000,4.000000,1.000000,4.100000,-104.099998,0.000000,0.400000
25%,257717.500000,99999.000000,1977.000000,4.000000,8.000000,38.074999,7.00000,29.600000,7.000000,1009.400024,...,6.000000,6.300000,7.000000,3.400000,7.000000,7.800000,19.000000,31.600000,0.000000,2.950000
50%,542105.000000,99999.000000,1990.000000,7.000000,16.000000,55.900002,8.00000,45.049999,8.000000,1014.400024,...,8.000000,9.500000,8.000000,5.800000,8.000000,11.100000,22.900000,46.900002,0.000000,6.300000
75%,725552.750000,99999.000000,2001.000000,10.000000,23.000000,71.199997,23.00000,57.700001,23.000000,1020.099976,...,23.000000,14.900000,23.000000,9.100000,23.000000,15.900000,28.900000,60.799999,0.000000,12.000000
max,999999.000000,99999.000000,2010.000000,12.000000,31.000000,101.800003,24.00000,79.500000,24.000000,1054.400024,...,24.000000,99.400002,24.000000,37.400002,24.000000,54.000000,64.900002,89.599998,12.200000,59.099998


### # 10 highest total_precipitation samples


In [27]:
df_gc.sort_values('total_precipitation', ascending=False).head(10)[['station_number', 'year', 'month', 'day', 'total_precipitation']]

,station_number,year,month,day,total_precipitation
1492,636940,1973,9,26,12.20
1659,913340,1989,4,17,6.89
1934,319930,1969,4,20,3.11
61,263180,1959,1,31,2.95
167,616300,1989,9,1,2.56
1314,723346,1991,12,3,2.17
429,726438,2002,7,7,2.04
1162,592980,1960,5,27,1.73
465,786500,1962,8,7,1.73
1147,421110,1962,8,24,1.69


# Sample code for USA SSN data
### This public dataset was created by the Social Security Administration and contains all names from Social Security card applications for births that occurred in the United States after 1879. 

In [0]:
import pandas as pd

sample_count = 2000
df = pd.io.gbq.read_gbq('''
  SELECT name, SUM(number) as count
  FROM `bigquery-public-data.usa_names.usa_1910_2013`
  WHERE state = 'TX'
  GROUP BY name
  ORDER BY count DESC
  LIMIT 100
''', project_id=project_id, verbose=False, dialect='standard')

df.head()

# Assignment
### Use the Google Political Advertisement analytics dataset to answer the following questions
### This data set contains information on how much is spent by verified advertisers on political advertising across Google Ad Services. In addition, insights on demographic targeting used in political ad campaigns by these advertisers are also provided, as are links to the actual political ad in the Google Transparency report.

### # Question 1. From the 'advertiser_stats table, list the top 20 advertisers and sort them by their total spending in USD 


In [0]:
# your code here

#### Question 2. From the 'geo spend' table, list the top 10 states by their total spending in USD

In [0]:
# your code here

#### Question 3. From the 'creative_stats' table, which advertiser had their ads running for the most number of days during the years 2016 and 2017

#### Question 4. From the 'campaign_targeting' table, list the top 5 advertisers who target younger population (18-24, 25-34, 35-44)

In [0]:
# your code here

#### Question 5. from the advertiser_stats, list the heaviest spending advertiser by calculating total_spending divided by total_creatives. For this you need to divide the total_spend_usd / total_creatives to get a ratio of their spending per ad. Sort the list and show only the top 10. 

In [0]:
# your code here